In [22]:
import numpy as np
from numpy.linalg import inv

Metode Inversi Linier \
inversi ini sekali jalan aja bro, sama seperti regresi linier sederhana. tidak perlu di looping

$$d = G m$$
keterangan: \
$d = $ data observasi \
$G = $ data kalkulasi \
$m = $ parameter model

$$m = [G^T G]^{-1} G^T d$$

Metode Inversi Non-linier \
Nah ini baru di looping bro, karena ada update parameter model setiap looping

$$d =  g(m)$$
keterangan: \
$d = $ data observasi (yang dioptimumkan *error*-nya) \
$g(m) = $ data kalkulasi (fungsi *forward modeling*) \
$m = $ parameter model (yang dicari)

**Pendekatan Lokal**

1.   Metode Gauss-Newton
$$m_{n+1} = m_n - [J_n^T J_n]^{-1} J_n^T \Delta d$$
2.   Metode Quasi-Newton
$$m_{n+1} = m_n - [J_n^T J_n + H_n^T \Delta d]^{-1} J_n^T \Delta d$$
3.   Metode Gradien
$$m_{n+1} = m_n - 2k \bigg( J_n^T \Delta d \bigg)$$
4.   Metode Levenberg-Marquardt
$$m_{n+1} = m_n + [J_n^T J_n + \varepsilon^2 I]^{-1} J_n^T \Delta d$$
5.   Metode Singular Value Decomposition

Note:
\begin{align}
\Delta d &= d^{cal} - d^{obs} \\
         &= g(m_n) - d^{obs}
\end{align}

turunan parsial Jacobian dan Hessian

\begin{align}
J = \frac{\delta g}{\delta m}
\end{align}

\begin{align}
H = \frac{\delta^2 g}{\delta m^2}
\end{align}

**Pendekatan Global** \
nah ini nanti akan dibagi menjadi algoritma stokastik dan detrministik
1.   Metode Algoritma Genetika (GA)
2.   Metode Simulated Annealing (SA)
3.   Metode Very Fast Simulated Annealing (VFSA)
4.   Metode Partical Swarm Optimization (PSO)
5.   Metode Flower Pollination Algorithm (FPA)
6.   Metode Fireflly Algorithm (FFA)
7.   Metode The Whale Optimization Algorithm $(WOA)^{23}$
8.   Metode Monte Carlo
9.   Metode Invers Random Search

In [3]:
def yi(a, b, xi):
  """
  a = parameter model [float]
  b = parameter model [float]
  xi = variabel bebas [array]

  return data kalkulasi [array]
  """
  return a + b * xi

def misfit(y_cal, y_obs):
  """
  y_cal = data kalkulasi [array]
  y_obs = data observasi [array]

  return nilai misfit [float]
  """
  N = len(y_cal)
  return np.sqrt(np.sum((y_cal - y_obs)**2) / N)

def model_param(x, y):
  """
  x = variabel bebas [array]
  y = data kalkulasi [array]

  return parameter model [array]
  """
  N = len(x)
  G = np.concatenate((np.ones((N, 1)), x.reshape(N, 1)), axis=1)
  d = y.reshape(N, 1)

  return np.linalg.inv(G.T @ G) @ G.T @ d

In [4]:
x = np.array([1,2,3,4,5,6,7,8,9,10])
y_obs = np.array([ 10.1,  20.1,  30.1,  40.1,  50.1,  60.1,  70.1,  80.1,  90.1, 100.1])

m = model_param(x, y_obs)
a, b = m.flatten()

y_cal = yi(a, b, x)

E = misfit(y_cal, y_obs)


print(f"a = {a}")
print(f"b = {b}")
print(f"RMSE = {E}")

a = 0.10000000000001563
b = 9.999999999999998
RMSE = 6.833781207288093e-15


Matriks Jacobian

$$J_{ij} = \frac{\delta g_i}{\delta m_j}$$

$$
J
=
\frac{dg}{d m}
=
\left[ \frac{\delta g}{\delta m_1}
\cdots \frac{\delta g}{\delta m_j}\right]
=
\begin{bmatrix}
\frac{\delta g_1}{\delta m_1} & \cdots &
\frac{\delta g_1}{\delta m_j} \\
\vdots & \ddots & \vdots \\
\frac{\delta g_i}{\delta m_1} & \cdots &
\frac{\delta g_i}{\delta m_j}
\end{bmatrix}
$$

In [5]:
# tentukan baris dan kolom matriks jacobian
# hitung f dari parameter tebakan
# hitung turunan parsial
def f(x, y, z=1):
  return (x + y) / z

data_observasi = [1,3,5,7]
x_tebakan = [1,2,3,4]
x_tebakan = np.array(x_tebakan, dtype=float)
y_tebakan = [1,2,3,4]
y_tebakan = np.array(y_tebakan, dtype=float)
z_tebakan = [1,2,3,4]
z_tebakan = np.array(z_tebakan, dtype=float)

tebakan = [x_tebakan, y_tebakan, z_tebakan]

jumlah_parameter = len(tebakan)
jumlah_data = len(data_observasi)

# inisialisasi matriks jacobian
jacobian = np.zeros((jumlah_data, jumlah_parameter))

data_kalkulasi = f(x_tebakan, y_tebakan, z_tebakan)

delta_x = 0.1
delta_y = 0.2
delta_z = 0.4

delta_m = [delta_x, delta_y, delta_z]

# persiapan forward finite different

baru = np.array([np.array(tebakan[i]) + delta_m[i] for i in range(jumlah_parameter)])

test = np.array([[tebakan[i] for i in range(jumlah_parameter)] for j in range(jumlah_parameter)])

for i in range(len(baru)):
  test[i, i] = baru[i]

# forward finite different
data_kalkulasi_2 = [f(*param) for param in test]

for kolom in range(jumlah_parameter):
  jacobian[:, kolom] = (data_kalkulasi_2[kolom] - data_kalkulasi) / delta_m[kolom]

print(jacobian)

[[ 1.          1.         -1.42857143]
 [ 0.5         0.5        -0.83333333]
 [ 0.33333333  0.33333333 -0.58823529]
 [ 0.25        0.25       -0.45454545]]


In [6]:
from copy import deepcopy as dc

def forward_difference(func1, arrays, h_list, y_initial, J):
  """
  Menghitung beda hingga maju untuk fungsi dengan parameter dinamis.
  """
  # Implementasi forward difference
  # ...
  for i in range(len(arrays)):
    # Forward modification
    modified_arrays_forward = [dc(arr) for arr in arrays]
    modified_arrays_forward[i] += h_list[i]
    y_forward = np.array([func1(*params) for params in zip(*modified_arrays_forward)])

    # Forward difference
    J[:, i] = (y_forward - y_initial) /  h_list[i]

  return J

def backward_difference(func1, arrays, h_list, y_initial, J):
  """
  Menghitung beda hingga mundur untuk fungsi dengan parameter dinamis.
  """
  # Implementasi backward difference
  # ...
  for i in range(len(arrays)):
    # Backward modification
    modified_arrays_backward = [dc(arr) for arr in arrays]
    modified_arrays_backward[i] -= h_list[i]
    y_backward = np.array([func1(*params) for params in zip(*modified_arrays_backward)])

    # Backward difference
    J[:, i] = (y_initial - y_backward) / h_list[i]

  return J

def central_difference(func1, arrays, h_list, J):
  """
  Menghitung beda hingga tengah untuk fungsi dengan parameter dinamis.
  """
  # Implementasi central difference
  # ...
  for i in range(len(arrays)):
    # Forward modification
    modified_arrays_forward = [dc(arr) for arr in arrays]
    modified_arrays_forward[i] += h_list[i]
    y_forward = np.array([func1(*params) for params in zip(*modified_arrays_forward)])

    # Backward modification
    modified_arrays_backward = [dc(arr) for arr in arrays]
    modified_arrays_backward[i] -= h_list[i]
    y_backward = np.array([func1(*params) for params in zip(*modified_arrays_backward)])

    # Central difference
    J[:, i] = (y_forward - y_backward) / (2 * h_list[i])
  return J

def jacobian(func1, *args, h_list, method="central"):
  """
  Menghitung matriks Jacobian dengan memilih metode perhitungan.
  Args:
  func1: Fungsi yang akan dihitung beda hingganya.
  *args: Daftar nilai untuk setiap parameter.
  h_list: Daftar langkah kecil (step sizes) untuk setiap parameter.
  method: Metode perhitungan ("forward", "backward", atau "central").

  Returns:
  tuple of numpy arrays: Hasil perhitungan beda hingga untuk fungsi pada daftar nilai parameter.
  """
  num_params = len(args)
  if num_params != len(h_list):
    raise ValueError("Jumlah parameter input dan langkah kecil harus sama.")

  # Mengonversi setiap daftar nilai menjadi numpy array
  arrays = [np.array(param) for param in args]

  # Mengecek panjang setiap array
  array_lengths = [len(arr) for arr in arrays]
  if not all(length == array_lengths[0] for length in array_lengths):
    raise ValueError("Semua parameter input harus memiliki panjang yang sama.")

  # Menghitung fungsi pada titik-titik awal
  y_initial = np.array([func1(*params) for params in zip(*arrays)])

  # Inisialisasi matriks Jacobian
  J = np.zeros((len(y_initial), num_params))

  if method == "forward":
    J = forward_difference(func1, arrays, h_list, y_initial, J)
  elif method == "backward":
    J = backward_difference(func1, arrays, h_list, y_initial, J)
  elif method == "central":
    J = central_difference(func1, arrays, h_list, J)
  else:
    raise ValueError("Metode perhitungan tidak valid. Gunakan 'forward', 'backward', atau 'central'.")

  return J

# forward modeling a.k.a g(m)
def f(x, y, z=2):
  return (x + y) / z

x_tebakan = np.array([1, 2, 3, 4], dtype=float)
y_tebakan = np.array([1, 2, 3, 4], dtype=float)
z_tebakan = np.array([1, 2, 3, 4], dtype=float)

x_values = x_tebakan
y_values = y_tebakan
z_values = z_tebakan

# params = [x_values, y_values]
params = [x_values, y_values, z_values]

# Langkah kecil untuk setiap parameter
delta = [0.1, 0.2, 0.4]
# delta = [0.1, 0.2]

# Hitung matriks jacobian
jacobain_matriks = jacobian(f, *params, h_list=delta, method="backward")
print(jacobain_matriks)

# Cetak hasilnya
for i, diff in enumerate(jacobain_matriks.T, start=1):
  print(f"Hasil beda hingga maju untuk parameter {i}:")
  print(diff)

[[ 1.          1.         -3.33333333]
 [ 0.5         0.5        -1.25      ]
 [ 0.33333333  0.33333333 -0.76923077]
 [ 0.25        0.25       -0.55555556]]
Hasil beda hingga maju untuk parameter 1:
[1.         0.5        0.33333333 0.25      ]
Hasil beda hingga maju untuk parameter 2:
[1.         0.5        0.33333333 0.25      ]
Hasil beda hingga maju untuk parameter 3:
[-3.33333333 -1.25       -0.76923077 -0.55555556]


# Tes dimensi matriks

In [18]:
d_obs = np.array([1.5,2.2,3.5,4.6,5.9])
d_cal = np.array([1,2,3,4,5])
m1 = np.array([1,2,3,4,5])
m2 = np.array([1,2,3,4,5])
m = np.array([m1, m2])

J = np.array([[1,2], 
              [1,2], 
              [3,2], 
              [8,9], 
              [5,6]])

delta_d = (d_obs - d_cal).reshape(-1, 1)
print(delta_d)

mmod = inv(J.T @ J) @ J.T @ delta_d
print(mmod)

new_m = m + mmod
print(new_m)

print(f"hasil invers:\n", inv(J.T @ J))


[[0.5]
 [0.2]
 [0.5]
 [0.6]
 [0.9]]
[[0.01432584]
 [0.08988764]]
[[1.01432584 2.01432584 3.01432584 4.01432584 5.01432584]
 [1.08988764 2.08988764 3.08988764 4.08988764 5.08988764]]
hasil invers:
 [[ 0.36235955 -0.31460674]
 [-0.31460674  0.28089888]]


# Tes rmse

In [20]:
def __rmse(delta_d):
		rmse = np.sqrt(np.sum(delta_d) ** 2 / len(delta_d))
		return rmse

rmse = __rmse(delta_d)
print(rmse)

1.2074767078498865
